# **KPMG Data Analytics Virtual Internship**

## Task 1
### Data Quality Assessment

    Assessment of data quality and completeness in preparation for analysis.

The client provided KPMG with 3 datasets:

* Customer Demographic 
* Customer Addresses
* Transactions data in the past 3 months

We decide to start the preliminary data exploration and identify ways to improve the quality of Sprocket Central Pty Ltd’s data.

# Data Exploration

## Import Library

In [1]:
# pip install openpyxl

In [2]:
# Data manipulation
import pandas as pd

In [3]:
# pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)

## Read Dataset

In [4]:
data = pd.ExcelFile('KPMG_Data_Analytics_Virtual_Internship_Dataset.xlsx')

In [5]:
# Read each data separately
Transactions = pd.read_excel(data, 'Transactions')
NewCustomerList = pd.read_excel(data, 'NewCustomerList')
CustomerDemographic = pd.read_excel(data, 'CustomerDemographic')
CustomerAddress = pd.read_excel(data, 'CustomerAddress')

## 1. Explore Transactions Data

In [6]:
# Dimensions of the DataFrame
Transactions.shape

(20000, 13)

In [7]:
print('Total rows: {}'.format(Transactions.shape[0]))
print('Total cols: {}'.format(Transactions.shape[1]))

Total rows: 20000
Total cols: 13


In [8]:
# Data types of the columns
Transactions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 13 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   transaction_id           20000 non-null  int64         
 1   product_id               20000 non-null  int64         
 2   customer_id              20000 non-null  int64         
 3   transaction_date         20000 non-null  datetime64[ns]
 4   online_order             19640 non-null  float64       
 5   order_status             20000 non-null  object        
 6   brand                    19803 non-null  object        
 7   product_line             19803 non-null  object        
 8   product_class            19803 non-null  object        
 9   product_size             19803 non-null  object        
 10  list_price               20000 non-null  float64       
 11  standard_cost            19803 non-null  float64       
 12  product_first_sold_date  19803 n

In [9]:
# Display a few rows
Transactions.sample(n=5, random_state=50)

,transaction_id,product_id,customer_id,transaction_date,online_order,order_status,brand,product_line,product_class,product_size,list_price,standard_cost,product_first_sold_date
7808,7809,80,2205,2017-09-21,0.0,Approved,OHM Cycles,Touring,low,medium,1073.07,933.84,35455.0
4702,4703,0,1944,2017-04-24,0.0,Approved,Trek Bicycles,Road,medium,medium,533.51,400.13,37823.0
8693,8694,46,3116,2017-10-04,NaN,Approved,Solex,Standard,low,medium,1289.85,74.51,39427.0
9085,9086,69,1356,2017-12-08,1.0,Approved,Norco Bicycles,Road,medium,large,1240.31,795.10,40553.0
16689,16690,78,1378,2017-04-26,1.0,Approved,Giant Bicycles,Standard,medium,large,1765.30,709.48,38193.0


- The column `online_order` should have an integer data type.
- The column `product_first_sold_date` should have a datetime data type.

In [10]:
# Detect missing value
Transactions.isna().any()

transaction_id             False
product_id                 False
customer_id                False
transaction_date           False
online_order                True
order_status               False
brand                       True
product_line                True
product_class               True
product_size                True
list_price                 False
standard_cost               True
product_first_sold_date     True
dtype: bool

In [11]:
# Summarize missing value
Transactions.isna().sum()

transaction_id               0
product_id                   0
customer_id                  0
transaction_date             0
online_order               360
order_status                 0
brand                      197
product_line               197
product_class              197
product_size               197
list_price                   0
standard_cost              197
product_first_sold_date    197
dtype: int64

In [12]:
Transactions.isna().sum().sum()

1542

- There exist 7 columns with null or missing values that may either be dropped or handled.
- Missing values in the `online_order` column can be filled with the mode value.

In [13]:
# Detect duplicated value
Transactions.duplicated().sum()

0

- There are no duplicate values, so the data is distinct/unique.

### Explore the columns

In [14]:
Transactions.columns

Index(['transaction_id', 'product_id', 'customer_id', 'transaction_date',
       'online_order', 'order_status', 'brand', 'product_line',
       'product_class', 'product_size', 'list_price', 'standard_cost',
       'product_first_sold_date'],
      dtype='object')

In [15]:
# Number of unique classes
Transactions.nunique()

transaction_id             20000
product_id                   101
customer_id                 3494
transaction_date             364
online_order                   2
order_status                   2
brand                          6
product_line                   4
product_class                  3
product_size                   3
list_price                   296
standard_cost                103
product_first_sold_date      100
dtype: int64

In [16]:
values = {'Feature Name'   : [],
          'Unique Classes' : []}

for col in Transactions.columns:
    values['Feature Name'].append(col)
    values['Unique Classes'].append(Transactions[col].unique())

values = pd.DataFrame(values)
values

,Feature Name,Unique Classes
0,transaction_id,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
1,product_id,"[2, 3, 37, 88, 78, 25, 22, 15, 67, 12, 5, 61, ..."
2,customer_id,"[2950, 3120, 402, 3135, 787, 2339, 1542, 2459,..."
3,transaction_date,"[2017-02-25 00:00:00, 2017-05-21 00:00:00, 201..."
4,online_order,"[0.0, 1.0, nan]"
5,order_status,"[Approved, Cancelled]"
6,brand,"[Solex, Trek Bicycles, OHM Cycles, Norco Bicyc..."
7,product_line,"[Standard, Road, Mountain, Touring, nan]"
8,product_class,"[medium, low, high, nan]"
9,product_size,"[medium, large, small, nan]"


In [17]:
# Convert the column from float to datetime
Transactions['product_first_sold_date'] = pd.to_datetime(Transactions['product_first_sold_date'], unit='s')
Transactions['product_first_sold_date'].sample(n=5, random_state=50)

7808    1970-01-01 09:50:55
4702    1970-01-01 10:30:23
8693    1970-01-01 10:57:07
9085    1970-01-01 11:15:53
16689   1970-01-01 10:36:33
Name: product_first_sold_date, dtype: datetime64[ns]

- The `product_first_sold_date` column has incorrect values as it shows everything happening the same day at different times.

### Descriptive Statistics

In [18]:
# Define features
num_tx = Transactions.select_dtypes(exclude = ['object'])
cat_tx = Transactions.select_dtypes(include = ['object'])

In [19]:
num_tx.shape[1]

8

In [20]:
cat_tx.shape[1]

5

#### 1. Numerical Features

In [21]:
# Summary statistics for numerical features
num_tx.describe().T

,count,mean,min,25%,50%,75%,max,std
transaction_id,20000.0,10000.5,1.0,5000.75,10000.5,15000.25,20000.0,5773.647028
product_id,20000.0,45.36465,0.0,18.0,44.0,72.0,100.0,30.75359
customer_id,20000.0,1738.24605,1.0,857.75,1736.0,2613.0,5034.0,1011.951046
transaction_date,20000,2017-07-01 14:08:05.280000,2017-01-01 00:00:00,2017-04-01 00:00:00,2017-07-03 00:00:00,2017-10-02 00:00:00,2017-12-30 00:00:00,NaN
online_order,19640.0,0.500458,0.0,0.0,1.0,1.0,1.0,0.500013
list_price,20000.0,1107.829449,12.01,575.27,1163.89,1635.3,2091.47,582.825242
standard_cost,19803.0,556.046951,7.21,215.14,507.58,795.1,1759.85,405.95566
product_first_sold_date,19803,1970-01-01 10:36:39.776549007,1970-01-01 09:14:19,1970-01-01 09:54:27,1970-01-01 10:36:56,1970-01-01 11:17:52,1970-01-01 11:51:50,NaN


#### 2. Categorical Features

In [22]:
# Summary statistics for categorical features
cat_tx.describe().T

,count,unique,top,freq
order_status,20000,2,Approved,19821
brand,19803,6,Solex,4253
product_line,19803,4,Standard,14176
product_class,19803,3,medium,13826
product_size,19803,3,medium,12990


## 2. Explore NewCustomerList Data

In [23]:
# Dimensions of the DataFrame
NewCustomerList.shape

(1000, 23)

In [24]:
print('Total rows: {}'.format(NewCustomerList.shape[0]))
print('Total cols: {}'.format(NewCustomerList.shape[1]))

Total rows: 1000
Total cols: 23


In [25]:
# Data types of the columns
NewCustomerList.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 23 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   first_name                           1000 non-null   object 
 1   last_name                            971 non-null    object 
 2   gender                               1000 non-null   object 
 3   past_3_years_bike_related_purchases  1000 non-null   int64  
 4   DOB                                  983 non-null    object 
 5   job_title                            894 non-null    object 
 6   job_industry_category                835 non-null    object 
 7   wealth_segment                       1000 non-null   object 
 8   deceased_indicator                   1000 non-null   object 
 9   owns_car                             1000 non-null   object 
 10  tenure                               1000 non-null   int64  
 11  address                        

In [26]:
# Drop the unnamed columns
NewCustomerList.drop(['Unnamed: 16', 'Unnamed: 17', 'Unnamed: 18', 'Unnamed: 19', 'Unnamed: 20'], axis=1, inplace=True)

In [27]:
# Display a few rows
NewCustomerList.sample(n=5, random_state=50)

,first_name,last_name,gender,past_3_years_bike_related_purchases,DOB,job_title,job_industry_category,wealth_segment,deceased_indicator,owns_car,tenure,address,postcode,state,country,property_valuation,Rank,Value
316,Audry,Fann,Female,3,1957-10-17,Pharmacist,Health,Mass Customer,N,Yes,15,19 Debs Parkway,3029,VIC,Australia,6,312,1.0200
459,Gregorius,Leal,Male,7,1951-04-29,NaN,Retail,Affluent Customer,N,Yes,21,66 Merry Court,2033,NSW,Australia,12,455,0.8925
854,Maurine,Clee,Female,45,1980-09-13,Automation Specialist II,Property,High Net Worth,N,Yes,5,6 Maple Plaza,2042,NSW,Australia,10,854,0.5610
11,Wheeler,Winward,Male,48,1999-08-30,Environmental Specialist,Manufacturing,Mass Customer,N,No,10,3 Golden Leaf Point,3216,VIC,Australia,8,12,1.6250
978,Artemis,Swanson,Male,77,1977-02-12,Web Designer II,Argiculture,Mass Customer,N,Yes,13,5 Melvin Park,3810,VIC,Australia,5,979,0.4165


- The `DOB` column should have a datetime data type.
- The `owns_car` should be converted to boolean data type.
- The `postcode` column is categorical data, rather than numerical data, so it should be converted to string data type.

In [28]:
# Detect missing value
NewCustomerList.isna().any()

first_name                             False
last_name                               True
gender                                 False
past_3_years_bike_related_purchases    False
DOB                                     True
job_title                               True
job_industry_category                   True
wealth_segment                         False
deceased_indicator                     False
owns_car                               False
tenure                                 False
address                                False
postcode                               False
state                                  False
country                                False
property_valuation                     False
Rank                                   False
Value                                  False
dtype: bool

In [29]:
# Summarize missing value
NewCustomerList.isna().sum()

first_name                               0
last_name                               29
gender                                   0
past_3_years_bike_related_purchases      0
DOB                                     17
job_title                              106
job_industry_category                  165
wealth_segment                           0
deceased_indicator                       0
owns_car                                 0
tenure                                   0
address                                  0
postcode                                 0
state                                    0
country                                  0
property_valuation                       0
Rank                                     0
Value                                    0
dtype: int64

In [30]:
NewCustomerList.isna().sum().sum()

317

- There are missing values in 4 columns, so they can be dropped or handled.
- Missing values in the `last_name` column can be filled with values in the `first_name` column.

In [31]:
# Detect duplicated value
NewCustomerList.duplicated().sum()

0

- There are no duplicate values.

### Explore the columns

In [32]:
NewCustomerList.columns

Index(['first_name', 'last_name', 'gender',
       'past_3_years_bike_related_purchases', 'DOB', 'job_title',
       'job_industry_category', 'wealth_segment', 'deceased_indicator',
       'owns_car', 'tenure', 'address', 'postcode', 'state', 'country',
       'property_valuation', 'Rank', 'Value'],
      dtype='object')

In [33]:
# Number of unique classes
NewCustomerList.nunique()

first_name                              940
last_name                               961
gender                                    3
past_3_years_bike_related_purchases     100
DOB                                     961
job_title                               184
job_industry_category                     9
wealth_segment                            3
deceased_indicator                        1
owns_car                                  2
tenure                                   23
address                                1000
postcode                                522
state                                     3
country                                   1
property_valuation                       12
Rank                                    324
Value                                   324
dtype: int64

In [34]:
values = {'Feature Name'   : [],
          'Unique Classes' : []}

for col in NewCustomerList.columns:
    values['Feature Name'].append(col)
    values['Unique Classes'].append(NewCustomerList[col].unique())

values = pd.DataFrame(values)
values

,Feature Name,Unique Classes
0,first_name,"[Chickie, Morly, Ardelis, Lucine, Melinda, Dru..."
1,last_name,"[Brister, Genery, Forrester, Stutt, Hadlee, Br..."
2,gender,"[Male, Female, U]"
3,past_3_years_bike_related_purchases,"[86, 69, 10, 64, 34, 39, 23, 74, 50, 72, 94, 4..."
4,DOB,"[1957-07-12, 1970-03-22, 1974-08-28 00:00:00, ..."
5,job_title,"[General Manager, Structural Engineer, Senior ..."
6,job_industry_category,"[Manufacturing, Property, Financial Services, ..."
7,wealth_segment,"[Mass Customer, Affluent Customer, High Net Wo..."
8,deceased_indicator,[N]
9,owns_car,"[Yes, No]"


- The `deceased_indicator` column has only one value or category, hence it can be dropped.

In [35]:
NewCustomerList['gender'].value_counts()

gender
Female    513
Male      470
U          17
Name: count, dtype: int64

In [36]:
NewCustomerList[NewCustomerList['gender'] == 'U'].reset_index()

,index,first_name,last_name,gender,past_3_years_bike_related_purchases,DOB,job_title,job_industry_category,wealth_segment,deceased_indicator,owns_car,tenure,address,postcode,state,country,property_valuation,Rank,Value
0,59,Normy,Goodinge,U,5,NaN,Associate Professor,IT,Mass Customer,N,No,4,7232 Fulton Parkway,3810,VIC,Australia,5,57,1.375000
1,226,Hatti,Carletti,U,35,NaN,Legal Assistant,IT,Affluent Customer,N,Yes,11,6 Iowa Center,2519,NSW,Australia,9,226,1.112500
2,324,Rozamond,Turtle,U,69,NaN,Legal Assistant,IT,Mass Customer,N,Yes,3,57025 New Castle Street,3850,VIC,Australia,3,324,1.010000
3,358,Tamas,Swatman,U,65,NaN,Assistant Media Planner,Entertainment,Affluent Customer,N,No,5,78 Clarendon Drive,4551,QLD,Australia,8,358,0.980000
4,360,Tracy,Andrejevic,U,71,NaN,Programmer II,IT,Mass Customer,N,Yes,11,5675 Burning Wood Trail,3030,VIC,Australia,7,361,0.977500
5,374,Agneta,McAmish,U,66,NaN,Structural Analysis Engineer,IT,Mass Customer,N,No,15,5773 Acker Way,4207,QLD,Australia,6,375,0.960000
6,434,Gregg,Aimeric,U,52,NaN,Internal Auditor,IT,Mass Customer,N,No,7,72423 Surrey Street,3753,VIC,Australia,5,433,0.906250
7,439,Johna,Bunker,U,93,NaN,Tax Accountant,IT,Mass Customer,N,Yes,14,3686 Waubesa Way,3065,VIC,Australia,6,436,0.903125
8,574,Harlene,Nono,U,69,NaN,Human Resources Manager,IT,Mass Customer,N,No,12,0307 Namekagon Crossing,2170,NSW,Australia,7,575,0.796875
9,598,Gerianne,Kaysor,U,15,NaN,Project Manager,IT,Affluent Customer,N,No,5,882 Toban Lane,2121,NSW,Australia,11,599,0.775000


- There are 17 rows where the gender is not specified.
- All customers with unspecified gender have none date of birth (DOB) data.

In [37]:
NewCustomerList['gender'] = NewCustomerList['gender'].replace('U', 'Unspecified')
NewCustomerList['gender'].value_counts()

gender
Female         513
Male           470
Unspecified     17
Name: count, dtype: int64

In [38]:
for col in NewCustomerList.columns[5:10]:    
    print(NewCustomerList[col].value_counts())
    print()

job_title
Associate Professor         15
Environmental Tech          14
Software Consultant         14
Chief Design Engineer       13
Assistant Manager           12
                            ..
Accountant II                1
Programmer IV                1
Administrative Officer       1
Accounting Assistant III     1
Web Developer I              1
Name: count, Length: 184, dtype: int64

job_industry_category
Financial Services    203
Manufacturing         199
Health                152
Retail                 78
Property               64
IT                     51
Entertainment          37
Argiculture            26
Telecommunications     25
Name: count, dtype: int64

wealth_segment
Mass Customer        508
High Net Worth       251
Affluent Customer    241
Name: count, dtype: int64

deceased_indicator
N    1000
Name: count, dtype: int64

owns_car
No     507
Yes    493
Name: count, dtype: int64



In [39]:
NewCustomerList['state'].value_counts()

state
NSW    506
VIC    266
QLD    228
Name: count, dtype: int64

In [40]:
NewCustomerList['country'].value_counts()

country
Australia    1000
Name: count, dtype: int64

### Descriptive Statistics

In [41]:
# Define features
num_nc = NewCustomerList.select_dtypes(exclude = ['object'])
cat_nc = NewCustomerList.select_dtypes(include = ['object'])

In [42]:
num_nc.shape[1]

6

In [43]:
cat_nc.shape[1]

12

#### 1. Numerical Features

In [44]:
# Summary statistics for numerical features
num_nc.describe().T

,count,mean,std,min,25%,50%,75%,max
past_3_years_bike_related_purchases,1000.0,49.836000,27.796686,0.00,26.750000,51.00,72.000,99.00000
tenure,1000.0,11.388000,5.037145,0.00,7.000000,11.00,15.000,22.00000
postcode,1000.0,3019.227000,848.895767,2000.00,2209.000000,2800.00,3845.500,4879.00000
property_valuation,1000.0,7.397000,2.758804,1.00,6.000000,8.00,9.000,12.00000
Rank,1000.0,498.819000,288.810997,1.00,250.000000,500.00,750.250,1000.00000
Value,1000.0,0.881714,0.293525,0.34,0.649531,0.86,1.075,1.71875


#### 2. Categorical Features

In [45]:
# Summary statistics for categorical features
cat_nc.describe().T

,count,unique,top,freq
first_name,1000,940,Rozamond,3
last_name,971,961,Sissel,2
gender,1000,3,Female,513
DOB,983,961,1965-07-03,2
job_title,894,184,Associate Professor,15
job_industry_category,835,9,Financial Services,203
wealth_segment,1000,3,Mass Customer,508
deceased_indicator,1000,1,N,1000
owns_car,1000,2,No,507
address,1000,1000,45 Shopko Center,1


## 3. Explore CustomerDemographic Data

In [46]:
# Dimensions of the DataFrame
CustomerDemographic.shape

(4000, 13)

In [47]:
print('Total rows: {}'.format(CustomerDemographic.shape[0]))
print('Total cols: {}'.format(CustomerDemographic.shape[1]))

Total rows: 4000
Total cols: 13


In [48]:
# Data types of the columns
CustomerDemographic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4000 entries, 0 to 3999
Data columns (total 13 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   customer_id                          4000 non-null   int64  
 1   first_name                           4000 non-null   object 
 2   last_name                            3875 non-null   object 
 3   gender                               4000 non-null   object 
 4   past_3_years_bike_related_purchases  4000 non-null   int64  
 5   DOB                                  3913 non-null   object 
 6   job_title                            3494 non-null   object 
 7   job_industry_category                3344 non-null   object 
 8   wealth_segment                       4000 non-null   object 
 9   deceased_indicator                   4000 non-null   object 
 10  default                              3698 non-null   object 
 11  owns_car                      

In [49]:
# Display a few rows
CustomerDemographic.sample(n=5, random_state=50)

,customer_id,first_name,last_name,gender,past_3_years_bike_related_purchases,DOB,job_title,job_industry_category,wealth_segment,deceased_indicator,default,owns_car,tenure
3218,3219,Denyse,Scutts,Female,94,1998-01-06 00:00:00,Web Developer III,Financial Services,Mass Customer,N,100,No,3.0
2007,2008,Jamill,Cudd,Male,77,1993-07-27 00:00:00,Analyst Programmer,Financial Services,Mass Customer,N,1,Yes,3.0
3639,3640,Merralee,Forman,Female,64,1985-04-28 00:00:00,Staff Scientist,NaN,Mass Customer,N,0ï¸â£ 1ï¸â£ 2ï¸â£ 3ï¸â£ 4ï¸â£ 5ï¸â£ 6ï¸â£ 7ï¸â...,No,10.0
2165,2166,Mareah,Woodnutt,Female,78,1984-11-09 00:00:00,Nurse,Property,Mass Customer,N,ð,No,16.0
2300,2301,Ken,Vashchenko,Male,66,1966-10-30 00:00:00,Senior Cost Accountant,Financial Services,Mass Customer,N,00ËÆ$-,Yes,19.0


- The column `tenure` should have an integer data type.

In [50]:
# Detect missing value
CustomerDemographic.isna().any()

customer_id                            False
first_name                             False
last_name                               True
gender                                 False
past_3_years_bike_related_purchases    False
DOB                                     True
job_title                               True
job_industry_category                   True
wealth_segment                         False
deceased_indicator                     False
default                                 True
owns_car                               False
tenure                                  True
dtype: bool

In [51]:
# Summarize missing value
CustomerDemographic.isna().sum()

customer_id                              0
first_name                               0
last_name                              125
gender                                   0
past_3_years_bike_related_purchases      0
DOB                                     87
job_title                              506
job_industry_category                  656
wealth_segment                           0
deceased_indicator                       0
default                                302
owns_car                                 0
tenure                                  87
dtype: int64

In [52]:
CustomerDemographic.isna().sum().sum()

1763

- There are missing values in 5 columns, therefore it is possible to drop or handle them.
- Missing values in the `last_name` column can be filled with values in the `first_name` column.

In [53]:
# Detect duplicated value
CustomerDemographic.duplicated().sum()

0

- There are no duplicate values.

### Explore the columns

In [54]:
CustomerDemographic.columns

Index(['customer_id', 'first_name', 'last_name', 'gender',
       'past_3_years_bike_related_purchases', 'DOB', 'job_title',
       'job_industry_category', 'wealth_segment', 'deceased_indicator',
       'default', 'owns_car', 'tenure'],
      dtype='object')

In [55]:
# Number of unique classes
CustomerDemographic.nunique()

customer_id                            4000
first_name                             3139
last_name                              3725
gender                                    6
past_3_years_bike_related_purchases     100
DOB                                    3448
job_title                               195
job_industry_category                     9
wealth_segment                            3
deceased_indicator                        2
default                                  90
owns_car                                  2
tenure                                   22
dtype: int64

In [56]:
values = {'Feature Name'   : [],
          'Unique Classes' : []}

for col in CustomerDemographic.columns:
    values['Feature Name'].append(col)
    values['Unique Classes'].append(CustomerDemographic[col].unique())

values = pd.DataFrame(values)
values

,Feature Name,Unique Classes
0,customer_id,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
1,first_name,"[Laraine, Eli, Arlin, Talbot, Sheila-kathryn, ..."
2,last_name,"[Medendorp, Bockman, Dearle, nan, Calton, Duck..."
3,gender,"[F, Male, Female, U, Femal, M]"
4,past_3_years_bike_related_purchases,"[93, 81, 61, 33, 56, 35, 6, 31, 97, 49, 99, 58..."
5,DOB,"[1953-10-12 00:00:00, 1980-12-16 00:00:00, 195..."
6,job_title,"[Executive Secretary, Administrative Officer, ..."
7,job_industry_category,"[Health, Financial Services, Property, IT, nan..."
8,wealth_segment,"[Mass Customer, Affluent Customer, High Net Wo..."
9,deceased_indicator,"[N, Y]"


In [57]:
CustomerDemographic['gender'].value_counts()

gender
Female    2037
Male      1872
U           88
F            1
Femal        1
M            1
Name: count, dtype: int64

In [58]:
# Renaming the categories
CustomerDemographic['gender'] = CustomerDemographic['gender'].replace({'F'     : 'Female',
                                                                       'Femal' : 'Female',
                                                                       'M'     : 'Male',
                                                                       'U'     : 'Unspecified'})
CustomerDemographic['gender'].value_counts()

gender
Female         2039
Male           1873
Unspecified      88
Name: count, dtype: int64

In [59]:
CustomerDemographic['default'].value_counts()

default
100                                       113
1                                         112
-1                                        111
-100                                       99
Ù¡Ù¢Ù£                                     53
                                         ... 
testâ testâ«                               31
/dev/null; touch /tmp/blns.fail ; echo     30
âªâªtestâª                                 29
ì¸ëë°í ë¥´                                 27
,ãã»:*:ã»ãâ( â» Ï â» )ãã»:*:ã»ãâ           25
Name: count, Length: 90, dtype: int64

- Column `default` has inconsistent values, so it can be dropped.

In [60]:
CustomerDemographic = CustomerDemographic.drop('default', axis=1)

### Descriptive Statistics

In [61]:
# Define features
num_cd = CustomerDemographic.select_dtypes(exclude = ['object'])
cat_cd = CustomerDemographic.select_dtypes(include = ['object'])

In [62]:
num_cd.shape[1]

3

In [63]:
cat_cd.shape[1]

9

#### 1. Numerical Features

In [64]:
# Summary statistics for numerical features
num_cd.describe().T

,count,mean,std,min,25%,50%,75%,max
customer_id,4000.0,2000.500000,1154.844867,1.0,1000.75,2000.5,3000.25,4000.0
past_3_years_bike_related_purchases,4000.0,48.890000,28.715005,0.0,24.00,48.0,73.00,99.0
tenure,3913.0,10.657041,5.660146,1.0,6.00,11.0,15.00,22.0


#### 2. Categorical Features

In [65]:
# Summary statistics for categorical features
cat_cd.describe().T

,count,unique,top,freq
first_name,4000,3139,Max,5
last_name,3875,3725,Pristnor,3
gender,4000,3,Female,2039
DOB,3913,3448,1978-01-30 00:00:00,7
job_title,3494,195,Business Systems Development Analyst,45
job_industry_category,3344,9,Manufacturing,799
wealth_segment,4000,3,Mass Customer,2000
deceased_indicator,4000,2,N,3998
owns_car,4000,2,Yes,2024


## 4. Explore CustomerAddress Data

In [66]:
# Dimensions of the DataFrame
CustomerAddress.shape

(3999, 6)

In [67]:
print('Total rows: {}'.format(CustomerAddress.shape[0]))
print('Total cols: {}'.format(CustomerAddress.shape[1]))

Total rows: 3999
Total cols: 6


In [68]:
# Data types of the columns
CustomerAddress.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3999 entries, 0 to 3998
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   customer_id         3999 non-null   int64 
 1   address             3999 non-null   object
 2   postcode            3999 non-null   int64 
 3   state               3999 non-null   object
 4   country             3999 non-null   object
 5   property_valuation  3999 non-null   int64 
dtypes: int64(3), object(3)
memory usage: 187.6+ KB


In [69]:
# Display a few rows
CustomerAddress.sample(n=5, random_state=50)

,customer_id,address,postcode,state,country,property_valuation
3218,3223,93723 Old Gate Hill,2021,NSW,Australia,9
2007,2012,33 Del Mar Junction,2264,NSW,Australia,8
3638,3643,686 Scoville Drive,3012,VIC,Australia,5
2165,2170,5 Muir Circle,2030,NSW,Australia,12
2300,2305,84 Main Lane,3149,VIC,Australia,10


- The `postcode` column is categorical data, rather than numerical data, so it should be converted to string data type.

In [70]:
# Detect missing value
CustomerAddress.isna().any()

customer_id           False
address               False
postcode              False
state                 False
country               False
property_valuation    False
dtype: bool

In [71]:
# Summarize missing value
CustomerAddress.isna().sum()

customer_id           0
address               0
postcode              0
state                 0
country               0
property_valuation    0
dtype: int64

In [72]:
CustomerAddress.isna().sum().sum()

0

- There are no missing values.

In [73]:
# Detect duplicated value
CustomerAddress.duplicated().sum()

0

- There are no duplicate values.

### Explore the columns

In [74]:
CustomerAddress.columns

Index(['customer_id', 'address', 'postcode', 'state', 'country',
       'property_valuation'],
      dtype='object')

In [75]:
# Number of unique classes
CustomerAddress.nunique()

customer_id           3999
address               3996
postcode               873
state                    5
country                  1
property_valuation      12
dtype: int64

In [76]:
values = {'Feature Name'   : [],
          'Unique Classes' : []}

for col in CustomerAddress.columns:
    values['Feature Name'].append(col)
    values['Unique Classes'].append(CustomerAddress[col].unique())

values = pd.DataFrame(values)
values

,Feature Name,Unique Classes
0,customer_id,"[1, 2, 4, 5, 6, 7, 8, 9, 11, 12, 13, 14, 15, 1..."
1,address,"[060 Morning Avenue, 6 Meadow Vale Court, 0 Ho..."
2,postcode,"[2016, 2153, 4211, 2448, 3216, 2210, 2650, 202..."
3,state,"[New South Wales, QLD, VIC, NSW, Victoria]"
4,country,[Australia]
5,property_valuation,"[10, 9, 4, 12, 8, 6, 7, 3, 5, 11, 1, 2]"


In [77]:
CustomerAddress['state'].value_counts()

state
NSW                2054
VIC                 939
QLD                 838
New South Wales      86
Victoria             82
Name: count, dtype: int64

In [78]:
CustomerAddress['state'] = CustomerAddress['state'].replace({'New South Wales' : 'NSW',
                                                             'Victoria'        : 'VIC'})
CustomerAddress['state'].value_counts()

state
NSW    2140
VIC    1021
QLD     838
Name: count, dtype: int64

In [79]:
CustomerAddress['country'].value_counts()

country
Australia    3999
Name: count, dtype: int64

- It seems that all columns contain accurate and consistent data.

### Descriptive Statistics

In [80]:
# Define features
num_ca = CustomerAddress.select_dtypes(exclude = ['object'])
cat_ca = CustomerAddress.select_dtypes(include = ['object'])

In [81]:
num_ca.shape[1]

3

In [82]:
cat_ca.shape[1]

3

#### 1. Numerical Features

In [83]:
# Summary statistics for numerical features
num_ca.describe().T

,count,mean,std,min,25%,50%,75%,max
customer_id,3999.0,2003.987997,1154.576912,1.0,1004.5,2004.0,3003.5,4003.0
postcode,3999.0,2985.755939,844.878364,2000.0,2200.0,2768.0,3750.0,4883.0
property_valuation,3999.0,7.514379,2.824663,1.0,6.0,8.0,10.0,12.0


#### 2. Categorical Features

In [84]:
# Summary statistics for categorical features
cat_ca.describe().T

,count,unique,top,freq
address,3999,3996,3 Mariners Cove Terrace,2
state,3999,3,NSW,2140
country,3999,1,Australia,3999
